## 🚀 Momentum Stock Screener Framework

### 📥 Setups

Installation and import of required packages

In [80]:
# !pip install -r requirements.txt

In [81]:
import yfinance as yf
import backtrader as bt

import pandas as pd
import numpy as np

import os

# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

Initialization of analysis parameters

In [82]:
TICKER: str = 'Nasdaq-100'
TABLE_NUM: int = 4  # For webscraping data
TABLE_COL: str = 'Ticker' # For webscraping data

START_DATE: str = '2019-01-01'
END_DATE: str = '2025-08-08'

### 🏗️ Data Acquisition

In [83]:
def fetch_stock_data(index: str=TICKER, table_num: int=TABLE_NUM, table_col: str=TABLE_COL, start_date: str=START_DATE, end_date: str=END_DATE) -> pd.DataFrame:
    """
    Fetch historical stock data for all tickers in the index, both saved as .csv and returned as a dictionary of DataFrames.

    Parameters:
        index (str): Wikipedia page name for the index (Default to TICKER).
        table_num (int): Wikipedia table number for the index (Default to TABLE_NUM).
        table_col (str): Wikipedia table column for the index (Default to TABLE_COL).
        start_date (str): Start date in 'YYYY-MM-DD' format (Default to START_DATE).
        end_date (str): End date in 'YYYY-MM-DD' format (Default to END_DATE).

    Returns:
        pd.DataFrame: OHLCV data for all tickers in the index.
    """
    url = f"https://en.wikipedia.org/wiki/{index}"

    try:
        ttable = pd.read_html(url)
    except Exception as e:
        raise ConnectionError(f"Failed to fetch tables from {url}: {e}")

    if table_num >= len(ttable) or table_col not in ttable[table_num].columns:
        raise ValueError(f"No table with recognizable ticker column found in {url}.")
    
    tickers = ttable[table_num][table_col].tolist()

    os.makedirs("data", exist_ok=True)
    data_dict = {}
    
    try:
        data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)
    except Exception as e:
        print(f"Error downloading")
    
    for ticker in tickers:
        df = data.xs(ticker, axis=1, level=1)
        df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

        if df.empty:
            print(f"Warning: No data for {ticker}")
        else:
            data_dict[ticker] = df

            file_path = os.path.join("data", f"{ticker}.csv")
            df.to_csv(file_path)

    return data_dict

### 📊 Stock Analysis

In [84]:
def check_breakout(price: float, bb_upper: float, bb_upper_prev: float) -> tuple[int, int]:
    """
    """
    return (int(price >= bb_upper), int(price < bb_upper_prev))

def check_uptrend(price: float, short_ema: float, short_ema_prev: float, long_ema: float, adx: float, adx_entry_threshold: float=25, adx_exit_threshold: float=20) -> tuple[int, int]:
    """
    """
    return (int(short_ema >= long_ema) + int(price >= short_ema and short_ema > short_ema_prev) + int(adx >= adx_entry_threshold), 
            int(short_ema < long_ema) + int(price < short_ema and short_ema <= short_ema_prev) + int(adx < adx_exit_threshold))

def check_momentum(rsi: float, macd: float, macd_signal: float, rsi_entry_threshold: int=60, rsi_exit_threshold: int=40) -> tuple[int, int]:
    """
    """
    return (int(rsi >= rsi_entry_threshold) + int(macd >= macd_signal) + int(macd > 0), 
            int(rsi <= rsi_exit_threshold) + int(macd < macd_signal) + int(macd <= 0))

def check_volume(vroc: float, entry_threshold: float=0.4, exit_threshold: float=0.1) -> tuple[int, int]:
    """
    """
    return (int(vroc >= entry_threshold), int(vroc <= exit_threshold))

In [85]:
def check_stop_loss(price: float, entries: list[tuple[float, int]], sl_threshold: float) -> tuple[int, list[tuple[float, int]]]:
    """
    """
    sl_indices = []
    sl_size = 0
    
    for i, (entry_price, size) in enumerate(entries):
        if price <= entry_price * (1 - sl_threshold):
            sl_indices.append(i)
            sl_size += size
    return sl_size, sl_indices

In [86]:
class MomentumStrategy(bt.Strategy):
    params = (('stop_loss_pct', 0.15), ('max_cash_frac', 0.6))  # 15% stop loss by default
       
    def __init__(self):
        self.bbands = bt.ind.BollingerBands(self.data.close, period=20, devfactor=1.5)

        self.ema_20 = bt.ind.EMA(self.data, period=20)
        self.ema_50 = bt.ind.EMA(self.data, period=50)
        self.macd = bt.ind.MACD(self.data.close)
        self.adx = bt.ind.ADX(self.data)

        self.rsi = bt.ind.RSI(period=14)

        self.vroc = bt.indicators.RateOfChange(self.data.volume, period=14)
        
        self.entry_lots = []  # List of (entry_price, size) tuples

        # Collect indicator values
        self.log = []

    def next(self):
        signal = 0  # +1 for long, 0 for hold, -1 for short
        price = self.datas[0].close[0]

        breakout_entry, breakout_exit = check_breakout(price, self.bbands.top[0], self.bbands.top[-1])
        uptrend_entry, uptrend_exit = check_uptrend(price, self.ema_20[0], self.ema_20[-1], self.ema_50[0], self.adx[0])
        momentum_entry, momentum_exit = check_momentum(self.rsi[0], self.macd.macd[0], self.macd.signal[0])
        volume_entry, volume_exit = check_volume(self.vroc[0])

        entry_score = breakout_entry + uptrend_entry + momentum_entry + volume_entry
        exit_score = breakout_exit + uptrend_exit + momentum_exit + volume_exit

        # Assumes only longing
        if not self.position and entry_score >= 5:  # Entry condition
            cash = self.broker.get_cash()
            position_value = cash * self.params.max_cash_frac * (entry_score / 8) 
            size = int(position_value/price)

            if size > 0:
                self.buy(size=size)
                self.entry_lots.append((price, size))
                signal = 1

        else:  # Exit condition
            stop_loss_size, stop_loss_indices = check_stop_loss(price, self.entry_lots, self.params.stop_loss_pct)
            
            if stop_loss_size > 0:
                self.sell(size=stop_loss_size)
                signal = -1

                for idx in reversed(stop_loss_indices):
                    self.entry_lots.pop(idx)

            if self.position.size > 0 and exit_score >= 3:
                self.close()
                self.entry_lots.clear()
                signal = -1

        # Store indicator values and signal
        self.log.append({
            'date': self.datas[0].datetime.date(0),
            'close': price,

            'bb_upper': self.bbands.top[0],
            'bb_middle': self.bbands.mid[0],
            'bb_lower': self.bbands.bot[0],

            'ema_20': self.ema_20[0],
            'ema_50': self.ema_50[0],
            'macd': self.macd.macd[0],
            'macd_signal': self.macd.signal[0],
            'macd_hist': self.macd.macd[0]-self.macd.signal[0],
            'adx': self.adx[0],

            'rsi': self.rsi[0],

            'vroc': self.vroc[0],

            'position': self.position.size,
            'signal': signal
        })

### Backtesting

In [87]:
def calculate_returns(df: pd.DataFrame) -> pd.DataFrame:
    """
    """
    entries = []
    returns = []

    for i in range(len(df)):
        row = df.iloc[i]
        price, date, trade_size = row['close'], row['date'], df['position'].shift(-1).iloc[i] - row['position']

        if trade_size > 0:  # Buy
            entries.append((date, price, trade_size))

        elif trade_size < 0:  # Sell
            size_to_sell = -trade_size

            while size_to_sell > 0 and entries:
                entry_date, entry_price, entry_size = entries.pop(0)
                match_size = int(min(size_to_sell, entry_size))
                pnl = (price - entry_price) * match_size

                returns.append({
                    'entry_date': entry_date,
                    'exit_date': date,
                    'entry_price': entry_price,
                    'exit_price': price,
                    'position_size': match_size,
                    'pnl': pnl,
                    'return_pct': (price - entry_price) / entry_price
                })

                if entry_size > match_size:  # Handle partial match
                    entries.insert(0, (entry_date, entry_price, entry_size - match_size))

                size_to_sell -= match_size

    return pd.DataFrame(returns)

In [89]:
# Load data from CSV
data = bt.feeds.GenericCSVData(
    dataname='/home/lawre/Finovax-Quantitative-Researcher-Internship-Summer-2025/Momentum-Stock-Screener-Week8/data/AAPL.csv',
    dtformat='%Y-%m-%d',
    timeframe=bt.TimeFrame.Days,
    compression=1,
    openinterest=-1,
    headers=True
)

# Create the backtest engine
cerebro = bt.Cerebro()

cerebro.addstrategy(MomentumStrategy)

# Add data
cerebro.adddata(data)

# Run backtest
results = cerebro.run()
strat = results[0]

backtest_log = pd.DataFrame(strat.log)
backtest_log.to_csv("backtest_log.csv", index=False)

In [92]:
res = calculate_returns(backtest_log)
res

,entry_date,exit_date,entry_price,exit_price,position_size,pnl,return_pct
0,2019-03-14,2019-03-29,43.994381,45.413005,119,168.816244,0.032246
1,2019-04-01,2019-04-02,45.846019,45.714452,82,-10.788461,-0.002870
2,2019-04-03,2019-04-04,46.231178,46.599587,82,30.209483,0.007969
3,2019-04-05,2019-04-15,46.996715,47.506280,97,49.427766,0.010843
4,2019-04-16,2019-04-26,47.716801,49.018212,79,102.811430,0.027274
...,...,...,...,...,...,...,...
115,2025-07-07,2025-07-08,212.679993,210.100006,24,-61.919678,-0.012131
116,2025-07-10,2025-07-11,210.509995,210.570007,24,1.440308,0.000285
117,2025-07-17,2025-07-18,210.570007,210.869995,24,7.199707,0.001425
118,2025-07-21,2025-07-22,212.100006,213.139999,24,24.959839,0.004903


In [94]:
(res['return_pct']>0).mean()

np.float64(0.45)